In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/av-wns-hack/test_aq1fgdb/test.csv
/kaggle/input/av-wns-hack/train_na17sgz/view_log.csv
/kaggle/input/av-wns-hack/train_na17sgz/item_data.csv
/kaggle/input/av-wns-hack/train_na17sgz/train.csv
/kaggle/input/av-wns-hack/sample_submission_ipsblct/sample_submission.csv


In [2]:
from datetime import datetime, timedelta 
import seaborn as sns
from tqdm import tqdm_notebook
import pdb

In [3]:
train = pd.read_csv('../input/av-wns-hack/train_na17sgz/train.csv')
test = pd.read_csv('../input/av-wns-hack/test_aq1fgdb/test.csv')
log = pd.read_csv('../input/av-wns-hack/train_na17sgz/view_log.csv')
item = pd.read_csv('../input/av-wns-hack/train_na17sgz/item_data.csv')

train['impression_time'] = pd.to_datetime(train['impression_time'])
test['impression_time'] = pd.to_datetime(test['impression_time'])
display(train.head(),test.head())

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0


,impression_id,impression_time,user_id,app_code,os_version,is_4G
0,a9e7126a585a69a32bc7414e9d0c0ada,2018-12-13 07:44:00,44754,127,latest,1
1,caac14a5bf2ba283db7708bb34855760,2018-12-13 07:45:00,29656,44,latest,0
2,13f10ba306a19ce7bec2f3cae507b698,2018-12-13 07:46:00,25234,296,latest,1
3,39c4b4dc0e9701b55a0a4f072008fb3f,2018-12-13 07:47:00,22988,207,latest,1
4,bf5a572cca75f5fc67f4b14e58b11d70,2018-12-13 07:48:00,35431,242,latest,1


In [4]:
### does not assume that no two time periods for the same 

def getRRByCategory(train,test,category,dvalueDiff,trace=False,limit=True):
    print("Category:{}".format(category))
    if limit:
        LL = limit
    else:
        LL = 'NoLimit'
        dvalueDiff = 0
        
    train['BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = -999
    test['BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = -999
    testMiss = [] ### for things that are not in test
    if trace:
            pdb.set_trace()
    for _uq in tqdm_notebook(train[category].unique()):
        _i = train[train[category]==_uq].index
        _i1 = test[test[category]==_uq].index
        _tt =  train.loc[_i,'impression_time']
        _tt1 = test.loc[_i1,'impression_time']
        #print("Starting test loop")
        for time in np.unique(_tt1.values):
            if limit:
                dd = pd.to_datetime(time).day
                increment = dd-12
                diff = pd.to_datetime(time)-timedelta(days=dvalueDiff+increment)
                IdxBool = (_tt>=diff) & (_tt<time)
            else:
                #print("aaaa")
                IdxBool = (_tt<time)
 
            IdxBoolffill = _tt1[_tt1==time].index
            
            try:
                sub = _tt[IdxBool]
                meanRRBefore = train.loc[sub.index,'is_click'].mean()
                if np.isnan(meanRRBefore):
                    meanRRBefore = -999
                test.loc[IdxBoolffill,'BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = meanRRBefore            
            except:
                meanRRBefore = -999
                test.loc[IdxBoolffill,'BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = meanRRBefore        
                testMiss.append(_uq)
        #print("Starting train loop")
        for time in np.unique(_tt.values):
            if limit:
                diff = pd.to_datetime(time)-timedelta(days=dvalueDiff)
                IdxBool = (_tt>=diff) & (_tt<time)
            else:
                IdxBool = (_tt<time)
 
            IdxBoolffill = _tt[_tt==time].index
            try:
                sub = _tt[IdxBool]
                meanRRBefore = train.loc[sub.index,'is_click'].mean()
                if np.isnan(meanRRBefore):
                    meanRRBefore = -999
            except:
                meanRRBefore = -999
                testMiss.append(_uq)
            train.loc[IdxBoolffill,'BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = meanRRBefore
        #print("-------")
    return train,test,testMiss

In [5]:
train.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0


In [6]:
train['app_code_os_version'] = train['app_code'].astype(str)+"_"+train['os_version']
test['app_code_os_version'] = test['app_code'].astype(str)+"_"+test['os_version']

train['user_id_app_code'] = train['user_id'].astype(str)+"_"+train['app_code'].astype(str)
test['user_id_app_code'] = test['user_id'].astype(str)+"_"+test['app_code'].astype(str)

In [7]:
for values in tqdm_notebook(['app_code_os_version','user_id_app_code']):
    train,test,testMissFinal = getRRByCategory(train,test,values,dvalueDiff=1,trace=False,limit=False)

Category:app_code_os_version


Category:user_id_app_code


In [8]:
### Next, create combination of categorical data and new stats

In [9]:
train.to_csv("trainWithRRstatsMore.csv",index=False)
test.to_csv("testWithRRstatsMore.csv",index=False)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 